# Deploy a finetuned model with AWS SageMaker

## Import nessecary packages

In [1]:
!pip install sagemaker --upgrade
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


## Initialize the HuggingFace model from S3 server or HuggingFace Hub

In [2]:
role = sagemaker.get_execution_role()

hub = {
  #'HF_MODEL_ID':'distilbert-base-uncased',          # model_id from hf.co/models
  'HF_TASK':'text-classification'                    # NLP task you want to use for predictions
}

huggingface_model = HuggingFaceModel(
    model_data = 's3://ml-model-tjn/models/model.tar.gz',
    py_version = 'py36',
    role = role,
    transformers_version = '4.6',
    pytorch_version = '1.7',
    env=hub,
)

## Deploy the model on a server and create an endpoint

In [3]:
predictor = huggingface_model.deploy(
    initial_instance_count = 1,
    instance_type = 'ml.m5.large')

----!

## Give inputs and make predictions

In [4]:
data = {
    "inputs": "Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market http://t.co/irWqCEZWEU"
}

predictor.predict(data)

[{'label': 'LABEL_1', 'score': 0.9932965636253357}]

## Delete the endpoint and release the resources (otherwise the bill will explode)

In [5]:
predictor.delete_endpoint()